**Greedy Algorithm for pricing**

In [2]:
import numpy as np

In [3]:
class Product:
    def __init__(self, price_vector, selected_price, to_be_rased ):
      self.price_vector = price_vector #all the possible prices of this product
      self.selected_price = selected_price
      self.test_price = selected_price #price index we are testing on
      self.to_be_rased = to_be_rased #boolean that is true if we need to rais this product price at this iteration

    def buy_or_not(prob):
      reward = np.random.binomial(1, prob) 
      return reward



In [4]:
class UserClass:
  def __init__(self,number_of_user, alfas, num_products_bought, p_matrix ) :
    #self.conversion_rate_matrix = np.random.uniform(0.0,1.0,(5,4))      #randomly generates the conversion rates for every price of every product (every row is a product, every column is a price)
    self.conversion_rate_matrix = np.array([[0.04, 0.01, 0.01, 0.81],
       [0.19, 0.07, 0.99, 0.3],
       [0.02, 0.10, 0.89, 0.02],
       [0.38 , 0.57, 0.01, 0.41],
       [0.32 , 0.92, 0.47 , 0.30]])
    self.number_of_user = number_of_user
    self.alfas = alfas
    self.num_products_bought = num_products_bought
    self.p_matrix = p_matrix

In [52]:
class Price_configuration:
  def __init__(self, reward, price_vector, products_list, p1_price_index,  p2_price_index,  p3_price_index,  p4_price_index,  p5_price_index ):
    self.reward= reward
    self.price_vector = price_vector #a vector of 5 elements containing the price selected for every product in this specific configuration
    self.price_vector_index = np.zeros(5)
    self.price_vector_index[0] = p1_price_index
    self.price_vector_index[1] = p2_price_index
    self.price_vector_index[2] = p3_price_index
    self.price_vector_index[3] = p4_price_index
    self.price_vector_index[4] = p5_price_index


In [78]:

def pull_price_vector( products_list, price_configuration_list ):

    k = 0
    for prod in products_list:
      if(prod.to_be_rased == 'false'):
        k += 1
    if k == 5 :
      for prod in products_list:
        prod.test_price = prod.selected_price

    #print('start pull function:')
    #receives a list of products and a price_configuration_list
    #iterate over all product in product list, raises the price of the product with to_be_rased = true
    else:
      for prod in products_list:
        prod.test_price = prod.selected_price
        if( prod.to_be_rased == 'true'):
          if(prod.selected_price < 3):
            prod.test_price = prod.selected_price + 1
          #print('test_price:',prod.test_price )

    i=0
    price_vector = np.zeros(5)
    for prod in products_list:
      price_vector[i] = prod.price_vector[int(prod.test_price)]
      i += 1
    #print('price_vector:', price_vector)
    #create a new price configuartion with reward = 0 and fills in the price_vectors with the prices to be tested and append it to price_configuration_list
    price_config = Price_configuration(0, price_vector, products_list, products_list[0].test_price, products_list[1].test_price, products_list[2].test_price, products_list[3].test_price, products_list[4].test_price)
    #print('price_config:', price_config)
    price_configuration_list.append(price_config)
    #lower the attribute to_be_rased at the current product and raise it in the next product unless we are at the last one 
    i=0
    if k != 5:
      for prod in products_list:
        if ( prod.to_be_rased == 'true'):
            prod.to_be_rased = 'false'
            if( i < 4):
              products_list[i+1].to_be_rased = 'true'
              break
        i += 1
    elif k == 5:
      products_list[0].to_be_rased = 'true'



def evaluate_reward( history, price_configuration):
    #recives the history and price_configuration
    tmp_sum = 0
    for product in history:
      for i in range(0,4):
        if product[i] == -1:
          product[i] = 0
        tmp_sum = tmp_sum + product[i]*price_configuration.price_vector[i]
    #calculate the reward and update it in the object price_configuartion
    price_configuration.reward = tmp_sum

def select_best_reward(price_configuration_list, products_list):
    #recives a list of price_configuration
    #takes the object with the max value of reward
    max_reward = 0
    winning_config = Price_configuration(0, np.zeros(5 ), products_list, 0 ,0, 0, 0, 0)
    for p_config in price_configuration_list:
      if p_config.reward > max_reward:
        max_reward = p_config.reward
        winning_config.reward = p_config.reward
        winning_config.price_vector = p_config.price_vector
        winning_config.price_vector_index = p_config.price_vector_index

    i = 0
    for prod in products_list:
      prod.to_be_rased = 'false'
      prod.selected_price = winning_config.price_vector_index[i]
      prod.test_price = prod.selected_price
      i += 1
    products_list[0].to_be_rased = 'true'
    return winning_config
    #from such an object pull the price vector and in each of the five objects product update the field selected_price

In [93]:
p_matrix = np.array([[0,0.5,0,0.5,0],
                      [0.5,0,0.5,0,0],
                      [0,0.5,0,0,0.5],
                      [0.5,0,0,0,0.5],
                      [0,0.5,0,0.5,0]], dtype = float)

num_products_bought = [1, 1, 1, 1, 1]

userClass1 = UserClass(10, np.random.dirichlet([1, 1, 1, 1, 1]), num_products_bought, p_matrix )
userClass2 = UserClass(20, np.random.dirichlet([1, 1, 1, 1, 1]), num_products_bought, p_matrix )
userClass3 = UserClass(50, np.random.dirichlet([1, 1, 1, 1, 1]), num_products_bought, p_matrix )

user_class_list = [userClass1, userClass2, userClass3]



price_vector = [10, 20, 30, 40]
p1 = Product(price_vector, 0, 'false' )
p2 = Product(price_vector, 0, 'false' )
p3 = Product(price_vector, 0, 'false' )
p4 = Product(price_vector, 0, 'false' )
p5 = Product(price_vector, 0, 'false' )

products_list = [p1, p2, p3, p4, p5]
  

price_configuration_list = []  


total_daily_users = userClass1.number_of_user + userClass2.number_of_user + userClass3.number_of_user


lambd = 0.5
n_step_max = 10
uc = np.random.choice(user_class_list, 1)[0]
initial_product = np.random.choice(5, 1, [a for a in uc.alfas])[0]




new_best_reward = 0
old_best_reward = -1


while new_best_reward > old_best_reward :

  
  old_best_reward = new_best_reward

  for i in range(5):
    history_for_price_config = []
    pull_price_vector(products_list, price_configuration_list)
    print( 'price config currently testing:', price_configuration_list[-1].price_vector)
    for j in range(0, total_daily_users):
      history_for_price_config.append(simulate_episode(uc.p_matrix, 10, initial_product, 0.5, uc.conversion_rate_matrix, price_configuration_list[-1].price_vector_index))   #returns every time that product has been bought
    evaluate_reward(history_for_price_config, price_configuration_list[-1])
    print('reward:',  price_configuration_list[-1].reward)
    #print( 'history:', history_for_price_config)

  new_best_reward = select_best_reward(price_configuration_list, products_list).reward
  print( 'best price config at this round:', select_best_reward(price_configuration_list, products_list).price_vector)
  print( 'new best reward this round:', new_best_reward)
  print( 'old best reward this round:', old_best_reward)
  print(' ')
  print(' ')
  

print( 'winning price config:', select_best_reward(price_configuration_list, products_list).price_vector)
print('winning_config', old_best_reward)

    






price config currently testing: [10. 10. 10. 10. 10.]
reward: 20.0
price config currently testing: [20. 10. 10. 10. 10.]
reward: 20.0
price config currently testing: [10. 20. 10. 10. 10.]
reward: 10.0
price config currently testing: [10. 10. 20. 10. 10.]
reward: 60.0
price config currently testing: [10. 10. 10. 20. 10.]
reward: 10.0
best price config at this round: [10. 10. 20. 10. 10.]
new best reward this round: 60.0
old best reward this round: 0
 
 
price config currently testing: [20. 10. 20. 10. 10.]
reward: 140.0
price config currently testing: [10. 20. 20. 10. 10.]
reward: 140.0
price config currently testing: [10. 10. 30. 10. 10.]
reward: 2160.0
price config currently testing: [10. 10. 20. 20. 10.]
reward: 80.0
price config currently testing: [10. 10. 20. 10. 20.]
reward: 60.0
best price config at this round: [10. 10. 30. 10. 10.]
new best reward this round: 2160.0
old best reward this round: 60.0
 
 
price config currently testing: [20. 10. 30. 10. 10.]
reward: 2170.0
price co

In [7]:
def simulate_episode(init_prob_matrix, n_steps_max, initial_product, lambd, conversion_rate_matrix, price_vector_index):
    prob_matrix = init_prob_matrix.copy()
    n_nodes = prob_matrix.shape[0]
    initial_active_nodes = np.zeros(5, dtype=int)
    initial_active_nodes[initial_product] = 1
    history = np.array([initial_active_nodes])
    buy_or_not_history = np.zeros(5, dtype=int)
    active_nodes = initial_active_nodes
    newly_active_nodes = active_nodes
    t=0

    while(t < n_steps_max and np.sum(newly_active_nodes) > 0):
        #print("giro ", t)
        #print(prob_matrix)

        #buy_or_not_nodes = buy_or_not( active_nodes, userClass, product_indx, price) 
        buy_or_not_nodes = np.zeros(5)
        for i, node in enumerate(active_nodes):
            if node == 1:
                random_sample = np.random.uniform(0.0, 1.0)
                bought = random_sample < conversion_rate_matrix[i][int(price_vector_index[i])]
                buy_or_not_nodes[i] = bought
                if bought: buy_or_not_history[i] = 1
                else: buy_or_not_history[i] = -1

        p = (prob_matrix.T * buy_or_not_nodes).T

        #print("p matrix : \n", p)
        #p is used to select from the prob matrix only the rows with active nodes  (.T compute the transpose)... returns the set of probabilities corresponding to the edges leaving from an active_node
        lambda_vector = np.zeros(n_nodes)
        lambda_matrix = np.zeros(shape = (n_nodes, n_nodes))
        for i in range(0, n_nodes):
            idx1 = -1
            idx2 = -1
            lambda_vector = np.zeros(5)
            while(idx1 == idx2 or idx1 == i or idx2 == i):
                [idx1, idx2] = np.random.choice(5, 2)
            lambda_vector[idx1] = 1
            lambda_vector[idx2] = lambd
            lambda_matrix[i] = lambda_vector
        #lambda matrix has primary products on the rows, secondary on the columns, 1 if the secondary is in the first slot, lambda otherwise 
        #print("lambda matrix : \n", lambda_matrix)
        p = (p * lambda_matrix)
        #print("p after lambda vec : \n", p)
        activated_edges = p > np.random.rand(p.shape[0], p.shape[1])
        #print("activated edges : \n", activated_edges)
        prob_matrix = prob_matrix * ((p!=0) == activated_edges)
        newly_active_nodes = (np.sum(activated_edges, axis=0) > 0) * (1 - active_nodes)
        active_nodes = np.array(active_nodes + newly_active_nodes)
        #print("active nodes at t+1: \n", active_nodes)
        history = np.concatenate((history, [newly_active_nodes]), axis = 0)
        #print("history : \n", history)
        t += 1

        #all edges leaving from the node that didn't buy go to 0
    
    return buy_or_not_history